In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("concrete_compressive_strength_data.csv")
df.info()

In [ ]:

df.head()

In [ ]:
df.isna().any()

In [ ]:
# Visualizing the different variables' relationships
sns.pairplot(data=df[['cement', 'water', 'age', 'superplastic', 'coarseagg', 'fineagg', 'strength']], plot_kws={'alpha':0.3, 's':20, 'color':'crimson'}, diag_kws={'color':'crimson'}, diag_kind='kde')

In [ ]:
# Adding features based on the variable relationships
df['log_strength'] = np.log(df['strength'])
df['log_water'] = np.log(df['water'])
df['log_age'] = np.log(df['age'])
df['log_superplastic'] = np.log(df['superplastic'])
df['water_cement_ratio'] = df['water'] / df['cement']

In [ ]:
# Checking the correlations of the chosen variables
df_corr = df.corr().round(2)

plt.figure(figsize=(10, 8))
sns.heatmap(df_corr, annot=True, annot_kws={'size':'10'}, vmax=1, vmin=-1, fmt='.1f')
plt.xticks(rotation=50, ha='right')

In [ ]:
df_corr['strength_absval'] = df_corr['strength'].abs()
df_corr[['strength', 'strength_absval']].sort_values(by='strength_absval', ascending=False)[1:]

In [ ]:
# Selecting predictors, target, and then splitting the data
from sklearn.model_selection import train_test_split
predictors = ['log_age', 'water', 'cement', 'superplastic']
target = ['strength']

X = df[predictors]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Joining the training data for outlier removal
Xy_train = X_train
Xy_train[target] = y_train

In [ ]:
# Checking the outliers visually
sns.boxplot(data=Xy_train)

In [ ]:
# Removing outliers in the training set for 1 iteration

# Defining outlier removal function
def remove_outliers(df, column):

    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3-Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return filtered_df


# Removing outliers in train set only
for i in range(0,len(predictors)):
  Xy_train = remove_outliers(Xy_train, predictors[i])

for j in range(0,len(target)):
  Xy_train = remove_outliers(Xy_train, target[j])

# Splitting to X_train_f and y_train_f
X_train_f = Xy_train[predictors]
y_train_f = Xy_train[target]

In [ ]:
# Checking the boxplot after removing outliers for 1 iteration
sns.boxplot(data=Xy_train)

In [ ]:
# Fitting the training data set, then plugging in the X_test to get y_pred
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train_f, y_train_f)

y_pred = reg.predict(X_test)

In [ ]:
# Plotting Predicted vs. Actual
y_test = np.array(y_test).flatten()
y_pred = y_pred.flatten()

residuals = y_test - y_pred
plt.figure(figsize=(10, 8))
sns.scatterplot(x=y_test, y=y_pred, color='black')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='crimson', linestyle='--')
plt.xlabel('Actual Strength')
plt.ylabel('Predicted Strength')
plt.title('Actual vs Predicted Strength')
plt.grid(True)
plt.show()

In [ ]:
# Plotting Residuals vs. Predicted
residuals = y_test - y_pred

plt.figure(figsize=(10, 8))
sns.scatterplot(x=y_pred, y=residuals, color='black')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted Strength')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Values')
plt.grid(True)
plt.show()

In [ ]:
# Calculating for model metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

In [ ]:
# Extracting the coefficients and intercept from the model
coefficients = reg.coef_
intercept = reg.intercept_

print("Intercept:", intercept)
print("Coefficients:", coefficients)

feature_names = X_train_f.columns

equation = f"y = {intercept[0]:.2f}"
for coef, name in zip(coefficients[0], feature_names):
    equation += f" + ({coef:.2f} × {name})"

print("Linear Regression Equation:")
print(equation)